# Bayesian Deep Learning: MC dropout

## 1. (Conventional) dropout

* ReLU는 약간 인지편향적인 감이 있고, GeLU 등으로 대체되는 추세임
* Dropout은 완전 무작위적으로 connection을 제거하니까...

`-` Fully Connected Network

* 모든 뉴런이 연결되어 있음
* 순차적 학습

`-` Dropout

* DNN의 업데이트 횟수는 매우 많음 $\to$ 업데이트 과정에서 학습의 방향을 바꾸자.
* DNN은 일반적으로 확률적 요소가 포함되어 있기 때문에 학습이 안정적. 수렴함.
* Dropout을 사용하면 step마다 완전 무작위로 뉴런의 연결을 끊어냄.
> Masking. 두 뉴런을 연결하지 않음.

`-` Stochastic masking

* 마스킹 확률: $1 - p$
* 일종의 앙상?블 관점으로 얘기할 수 있었는데, 일반적인 앙상블이라고 말하긴 힘들어.

1. Activated parameter들이 계속 바뀜(weights and biases). 마스킹된 파라미터는 스텝마다 다름
2. 추론 시, 업데이트된 파라미터의 뒷부분 스텝들의 평균을 사용한다. 마지막 업데이트가 전체 파라미터를 대변하지 않는다.
3. 

> 최종적으로 각 파라미터들은 업데이트된 횟수가 다름. 이 경우 평가 시에 가중치는 훈련 뒷부분 몇 개의 step에서 평균적인 값으로 적용함.

`-` Test procedure

* 최종 평가 단계에서 마스킹되지 않을 확률을 곱해준다.
* Conventional ensemble이 아니다: 각 모형이 독립적이지 않음
* Boosting도 아니다: 마스킹은 완전 확률적으로 수행. 이전 스텝에서의 정보를 반영하지 않음. $\to$ DNN은 앙상블할 필요가 없이 Dropout만으로 충분하다.
* 훈련 스텝마다 다른 네트워크를 사용한다.

`-` Dropout의 특징

* dropout을 사용하면 total step만큼 파라미터가 갱신되지 않는다. 따라서 에폭을 증가시켜줄 필요가 있다.
* 그래디언트의 흐름을 더 부드럽게 만들어준다. vanishing/exploding을 막아준다.

## 2. Bayesian DL, MC dropout